# Parallelization of Simulation and Optimization

In [4]:
from qopt import *
import numpy as np
import time

n_time_steps = 200
time_step = 1  # ns
oversampling = 100
n_traces = 10

delta_omega_0 = 10e-3 * 2 * np.pi  # GHz = 10 * 2 pi Mhz


transfer_function = OversamplingTF(oversampling=oversampling, num_ctrls=2)
noise_trace_generator = NTGQuasiStatic(
    standard_deviation=[delta_omega_0, ] * 2,
    n_samples_per_trace=n_time_steps * oversampling,
    n_traces=n_traces,
)

solver = SchroedingerSMCControlNoise(
    transfer_function=transfer_function,
    h_ctrl=[DenseOperator.pauli_x(), DenseOperator.pauli_y()],
    h_drift=[DenseOperator.pauli_z() * 0],
    tau=time_step * np.ones(n_time_steps),
    noise_trace_generator=noise_trace_generator,
    calculate_propagator_derivatives=False
)

solver.set_optimization_parameters(np.random.rand(n_time_steps, 2))

In [5]:
start = time.time()
solver._compute_dyn_gen_noise()
end = time.time()
print(end - start)

34.81517791748047


In [6]:
start = time.time()
_ = solver._compute_propagation(calculate_propagator_derivatives=False,
                                processes=1)
end = time.time()
print(end - start)

119.2195212841034


In [7]:
start = time.time()
_ = solver._compute_propagation(calculate_propagator_derivatives=False,
                       processes=None)
end = time.time()
print(end - start)

55.877259254455566


In [ ]:
from multiprocessing import pool